<center>
    <img src="https://image.shutterstock.com/image-vector/regression-concept-keywords-people-icons-260nw-1820291330.jpg" width="600" alt="cognitiveclass.ai logo"  />
</center>

# Multiple Linear Regression

In [1]:
import pandas as pd 
import graphing
df = pd.read_excel('Data Collection\Petrol.xlsx', sheet_name='Combinedv2')

df1 = pd.read_excel('Data Collection\BFP.xlsx', sheet_name='Combined')
df1['Crude_Rands'] = (df1.Avg_Brent_Crude * df1.Ex_Rate)

#New dataset with new columns
to_drop = df[['BFP ', 'Fuel tax', 'Road accident fund']]
data = df.drop(to_drop, axis=1)
#Create a new column of Other cost
data['Other_cost']=data.iloc[:,2:13].sum(axis=1)
data_other = data['Other_cost']

# merge all into one dataset
all_df = [df1, to_drop, data_other]
dataset = pd.concat(all_df, axis=1)

#Drop duplicate BFP Column
dataset = dataset.drop(['BFP '], axis=1)
dataset.head()

,Date,BFP,Ex_Rate,Avg_Brent_Crude,Petrol,Crude_Rands,Fuel tax,Road accident fund,Other_cost
0,2010-01-01,406.263,7.5246,74.31,786.0,559.153026,150.0,64,161.737
1,2010-02-01,424.263,7.4735,76.84,804.0,574.263740,150.0,64,161.737
2,2010-03-01,430.563,7.6902,73.17,810.3,562.691934,150.0,64,161.737
3,2010-04-01,453.063,7.4753,78.89,858.3,589.726417,167.5,72,161.737
4,2010-05-01,465.063,7.3749,85.75,871.8,632.397675,167.5,72,163.237


#### Data Visualization

In [2]:
import plotly.express as px

fig = px.scatter(dataset, x ="Avg_Brent_Crude",y= "Petrol", title='Crude Oil vs Petrol')
fig1 = px.scatter(dataset, x ="Ex_Rate", y= "Petrol", title='Exchange Rate vs Petrol')

fig.show()
fig1.show()

In [3]:
import plotly.express as px
fig = px.scatter(dataset, x='Avg_Brent_Crude', y='Petrol', trendline='ols', trendline_color_override='red', title='Crude Oil vs Petrol')

fig1 = px.scatter(dataset, x='Ex_Rate', y='Petrol', trendline='ols', trendline_color_override='red', title='Exchange Rate vs Petrol')

fig.show()
fig1.show()


### Simple Linear Regression: Microsoft

In [4]:
import statsmodels.formula.api as smf
import graphing # custom graphing code from Microsft

for feature in ["Avg_Brent_Crude", "Ex_Rate"]:
    # Perform linear regression. This method takes care of
    # the entire fitting procedure for us.
    formula = "Petrol ~ " + feature
    simple_model = smf.ols(formula = formula, data = dataset).fit()

    print(feature)
    print("R-squared:", simple_model.rsquared)
    
    # Show a graph of the result
    graphing.scatter_2D(dataset, label_x=feature, 
                                 label_y="Petrol",
                                 title = feature,
                                 trendline=lambda x: simple_model.params[1] * x + simple_model.params[0],
                                 show=True)

Avg_Brent_Crude
R-squared: 0.0474499912561569


Ex_Rate
R-squared: 0.5410971695838112


### R-Squared

In [14]:
formula = "Petrol ~ Ex_Rate"
petrol_trained_model = smf.ols(formula = formula, data = dataset).fit()
petrol_naive_model = smf.ols(formula = formula, data = dataset).fit()
petrol_naive_model.params[0] = dataset['Petrol'].mean()
petrol_naive_model.params[1] = 0

print("naive R-squared:", petrol_naive_model.rsquared)
print("trained R-squared:", petrol_trained_model.rsquared)

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="Ex_Rate", 
                                label_y="Petrol",
                                title = "Naive model",
                                trendline=lambda x: dataset['Petrol'].mean().repeat(len(x)), 
                                show=True)
# Show a graph of the result
graphing.scatter_2D(dataset, label_x="Ex_Rate", 
                                label_y="Petrol",
                                title = "Trained model",
                                trendline=lambda x: petrol_trained_model.params[1] * x + petrol_trained_model.params[0])



naive R-squared: 2.220446049250313e-16
trained R-squared: 0.5410971695838112


In [15]:
model = smf.ols(formula = "Petrol ~ Avg_Brent_Crude + Ex_Rate", data = dataset).fit()

print("R-squared:", model.rsquared)

R-squared: 0.8566876783297591


In [7]:
import numpy as np
# Show a graph of the result
# this needs to be 3D, because we now have three variables in play: two features and one label

def predict( Avg_Brent_Crude , Ex_Rate):
    '''
    This converts given Avg_Brent_Crude and Ex_Change values into a prediction from the model
    '''
    # to make a prediction with statsmodels, we need to provide a dataframe
    # so create a dataframe with just the Avg_Brent_Crude and Ex_Rate variables
    df = pd.DataFrame(dict(Avg_Brent_Crude=[Avg_Brent_Crude], Ex_Rate=[Ex_Rate]))
    return model.predict(df)

# Create the surface graph
fig = graphing.surface(
    x_values=np.array([min(dataset.Avg_Brent_Crude), max(dataset.Avg_Brent_Crude)]),
    y_values=np.array([0, 25]),
    calc_z=predict,
    axis_title_x="Brent Crude Oil",
    axis_title_y="Exchange Rate",
    axis_title_z="Petrol"
)

# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.Avg_Brent_Crude, y=dataset.Ex_Rate, z=dataset.Petrol, mode='markers')
fig.show()

In [8]:
# Print summary information
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Petrol   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.855
Method:                 Least Squares   F-statistic:                     430.4
Date:                Mon, 14 Mar 2022   Prob (F-statistic):           1.79e-61
Time:                        10:50:16   Log-Likelihood:                -891.55
No. Observations:                 147   AIC:                             1789.
Df Residuals:                     144   BIC:                             1798.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -880.9382     87.955    -10.016      0.000   -1054.789    -707.088
Avg_Brent_Crude     9.3756      0.526     17.807      0.000       8.335      10.416
Ex_Rate           126.0346      4.420     28.515      0.000     117.298     134.771
==============================================================================
Omnibus:                       11.083   Durbin-Watson:                   0.256
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               13.895
Skew:                          -0.478   Prob(JB):                     0.000961
Kurtosis:                       4.164   Cond. No.                         822.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
petrol_trained_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Petrol   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     7.223
Date:                Mon, 14 Mar 2022   Prob (F-statistic):            0.00804
Time:                        10:50:16   Log-Likelihood:                -1030.8
No. Observations:                 147   AIC:                             2066.
Df Residuals:                     145   BIC:                             2072.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1509.5092     68.392     22.071      0.000    1374.335    1644.683
Avg_Brent_Crude    -2.2888      0.852     -2.688      0.008      -3.972      -0.606
==============================================================================
Omnibus:                        4.159   Durbin-Watson:                   0.068
Prob(Omnibus):                  0.125   Jarque-Bera (JB):                3.765
Skew:                           0.292   Prob(JB):                        0.152
Kurtosis:                       3.523   Cond. No.                         246.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Using Sklearn

Creat train and test dataset

In [16]:
cdf = dataset[['Avg_Brent_Crude', 'Ex_Rate', 'Petrol']]

In [20]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

Mulltiple Regression Model

In [21]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['Avg_Brent_Crude', 'Ex_Rate']])
y = np.asanyarray(train[['Petrol']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[  9.66428564 130.22517033]]


Prediction

In [22]:
y_hat= regr.predict(test[['Avg_Brent_Crude', 'Ex_Rate']])
x = np.asanyarray(test[['Avg_Brent_Crude', 'Ex_Rate']])
y = np.asanyarray(test[['Petrol']])
print("RSS: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('R-Squared: %.2f' % regr.score(x, y))

RSS: 12340.25
R-Squared: 0.83


C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but LinearRegression was fitted without feature names

